# qacsv SIN pandas

Podrías reescribri el código anterior de manera que haga lo mismo pero sin utilizar la librería pandas?

Claro, puedo ayudarte a reescribir el código para que no dependa de `pandas`. Usaremos `openpyxl` para leer el archivo Excel y `csv` para guardar los resultados. Este reemplazo sigue el mismo flujo general: leer el Excel, armar las consultas SQL reemplazando las fechas, ejecutarlas en SQL Server con `pyodbc`, y guardar los resultados como CSV.

---

### ✅ **Versión sin `pandas` (usando `openpyxl` y `csv`)**

A continuación te presento una versión adaptada:

```python
import os
import pyodbc
import openpyxl
import csv
from datetime import datetime

# Parámetros
EXCEL_PATH = 'queries.xlsx'
OUTPUT_DIR = 'resultados'
f_corte = datetime.today().strftime('%Y%m%d')
fcorte_12 = (datetime.today().replace(day=1) - timedelta(days=1)).strftime('%Y%m%d')

# Crear carpeta de salida
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Cargar Excel
wb = openpyxl.load_workbook(EXCEL_PATH, data_only=True)
ws = wb.active

# Leer encabezados
headers = [cell.value for cell in ws[1]]

# Obtener el índice de las columnas
id_idx = headers.index('id_nm')
query_idx = headers.index('Modo de armado')
cnx_idx = headers.index('conn')

# Iterar sobre las filas del Excel
for row in ws.iter_rows(min_row=2, values_only=True):
    id_nm = row[id_idx]
    raw_query = row[query_idx]
    conn_str = row[cnx_idx]

    if not raw_query or not conn_str:
        print(f"{id_nm}: Saltado por falta de datos")
        continue

    # Reemplazos de fechas
    query = raw_query.replace('f_corte', f_corte).replace('fcorte_12', fcorte_12)

    # Verificación básica
    if 'select' not in query.lower():
        print(f"{id_nm}: Consulta inválida")
        continue

    # Conexión a la base
    try:
        cnxn = pyodbc.connect(conn_str)
        cursor = cnxn.cursor()
        cursor.execute(query)
        columns = [desc[0] for desc in cursor.description]
        rows = cursor.fetchall()

        # Guardar en CSV
        output_path = os.path.join(OUTPUT_DIR, f"{id_nm}.csv")
        with open(output_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(columns)
            writer.writerows(rows)

        print(f"{id_nm}: OK - Guardado {len(rows)} filas")
        cursor.close()
        cnxn.close()

    except Exception as e:
        print(f"{id_nm}: ERROR al ejecutar la consulta\n{query}\n{e}")
```

---

### 📌 Notas clave:

* Usamos `openpyxl` para leer el archivo Excel sin `pandas`.
* Se hace el reemplazo de `f_corte` y `fcorte_12` manualmente en las cadenas.
* Se conecta con `pyodbc`, ejecuta la consulta y guarda los resultados con la librería `csv`.
* Incluye validación mínima para que las consultas tengan `SELECT`.

¿Querés que esta versión también registre un log por archivo con los errores?


El error que encontrás:

```
raise error from None
pyodbc.ProgrammingError: ('42000', "[42000] [Microsoft][Controlador ODBC SQL Server][SQL Server]Error de sintaxis cerca de la palabra clave 'and'. (156) (SQLExecDirectW)")
```

indica que en alguna de las consultas SQL que estás intentando ejecutar hay un **error de sintaxis**, específicamente cerca de la palabra clave **`AND`**. Este tipo de error suele darse por alguno de estos motivos:

---

### 🔍 **Causas comunes del error:**

1. **Falta de condición previa al `AND`:**

   ```sql
   SELECT * FROM tabla WHERE AND campo = 'valor'
   ```

   El `AND` no puede ir al inicio de la cláusula `WHERE`.

2. **Uso incorrecto de comillas o caracteres especiales:**
   Algunos archivos `.xlsx` exportados desde LibreOffice (o Google Sheets) pueden introducir comillas raras o espacios invisibles.

3. **Error en la plantilla de query en la columna `"Modo de armado"` del Excel**:
   Tu código reemplaza `"f_corte"` y `"fcorte_12"` por fechas dentro de las cadenas de la consulta. Si alguna consulta queda mal formada, fallará.

---

### ✅ **Pasos para depurar:**

1. **Agregar `print(qry)` justo antes del `assert` en la línea:**

   ```python
   assert 'select' in qry.lower()
   ```

   Esto te va a mostrar el texto completo de la consulta que está generando el error.

2. **Corroborar si la query empieza bien:**
   Revisá si hay un `AND` flotante, o algo como:

   ```sql
   WHERE AND fecha = '20250430'
   ```

3. **Guardar la query con error en el log y mostrarla claramente:**
   Ya estás haciendo algo de eso con:

   ```python
   ln = f'''{csv.id_nm}ERROR en 10.d! Falló  Query a la DB
       {e}'''
   ```

   Pero podrías agregarle también la consulta problemática para que el mensaje de error sea más útil:

   ```python
   ln = f'''{csv.id_nm}ERROR en 10.d! Falló  Query a la DB:
       {qry}
       {e}'''
   ```

4. **Validar que los reemplazos `.str.replace()` de las fechas no estén generando problemas:**

   Si tenés:

   ```python
   lst_df['Modo de armado'] = lst_df['Modo de armado'].str.replace('f_corte', f_corte)
   ```

   Y `f_corte = '20250430'`, fijate si la cadena resultante queda:

   ```sql
   ... AND fecha = '20250430'
   ```

   Pero si ya había comillas en la cadena del Excel (`'f_corte'`), podrías terminar con algo incorrecto como:

   ```sql
   ... AND fecha = ''20250430''
   ```

   Solución: hacé el replace sin las comillas simples en `f_corte`, y asegurate de que estén puestas correctamente en el texto original.

---

### 📌 ¿Cómo seguir?

Mostrame uno o dos ejemplos de valores de la columna `"Modo de armado"` en el Excel (antes y después del reemplazo si podés), así revisamos cómo están formadas las consultas.

¿Querés que te prepare un código para loguear las queries antes de ejecutarlas para que puedas identificar rápidamente cuál está fallando?
